In [29]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
yf.pdr_override()
from pandas_datareader import data as pdr
import pandas_market_calendars as mcal
bse = mcal.get_calendar('BSE')

In [30]:
c = ['ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BPCL',
 'BHARTIARTL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDUNILVR',
 'HDFC',
 'ICICIBANK',
 'ITC',
 'IOC',
 'INDUSINDBK',
 'INFY',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NTPC',
 'NESTLEIND',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBILIFE',
 'SHREECEM',
 'SBIN',
 'SUNPHARMA',
 'TCS',
 'TATAMOTORS',
 'TATASTEEL',
 'TECHM',
 'TITAN',
 'UPL',
 'ULTRACEMCO',
 'WIPRO']
adder = lambda x: x+".NS"
c = [adder(i) for i in c]
nyse = mcal.get_calendar('XBOM')
bse = nyse.valid_days(start_date = dt.date.today()-dt.timedelta(days=59), end_date=dt.date.today())

In [31]:
def download_data(companies):
    dataset = pdr.get_data_yahoo(companies,
                                  start = dt.date.today()-dt.timedelta(days=59),
                                  end   = dt.date.today(),
                                  #start = str(dt.date.today()),
                                  interval = "5m",
                                  group_by ='ticker'
                                 )
    return dataset
data  = download_data(c)

[*********************100%***********************]  50 of 50 completed


In [32]:
def avg_cumulative_vol(vol):
    x = np.empty((5,75), dtype=int)
    for i in range(0,5):
        x[i][:] =vol[75*i : (75*i)+75]
    y = x.cumsum(axis=1)
    y = y.sum(axis=0)/5
    return y
def cumulative_vol(vol):
    vol = vol[0:75].values
    y   = vol.cumsum()
    return y

## Data cleaning

In [33]:
for d in range(0,34):
    if(data.index.date[d*75] == data.index.date[d*75+74]):
        continue
    else:
        k         = data.index.tolist()
        up_date   = k.index(data.index[d*75]+dt.timedelta(days=1))
        da = k.index(data.index[d*75])
        data.drop(k[da : up_date],inplace=True)

In [35]:
relativeVol = pd.DataFrame()
for j in c:
    vol = data[j]["Volume"]
    relvol    = []
    for i in range(0,vol.shape[0]//75-5):
        x = avg_cumulative_vol(vol[i*75 : i*75+75*5])
        y = cumulative_vol(vol[(i+5)*75 : (i+5)*75+75])
        relative_vol = y/x
        relvol.extend(relative_vol)
        k = list(np.zeros(375,dtype=int))
        k.extend(relvol)
    relativeVol[j] = k

<ipython-input-35-bf6156e2862b>:8: RuntimeWarning: invalid value encountered in true_divide
  relative_vol = y/x
<ipython-input-35-bf6156e2862b>:8: RuntimeWarning: divide by zero encountered in true_divide
  relative_vol = y/x
